In [1]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [2]:
!kaggle competitions download -c "homesite-quote-conversion"

  0% 0.00/258k [00:00<?, ?B/s]
100% 258k/258k [00:00<00:00, 38.6MB/s]
 89% 33.0M/37.1M [00:02<00:00, 6.73MB/s]
100% 37.1M/37.1M [00:02<00:00, 17.5MB/s]
 36% 9.00M/24.7M [00:01<00:03, 5.48MB/s]
100% 24.7M/24.7M [00:01<00:00, 16.5MB/s]


In [3]:
! unzip train.csv.zip -d train
! unzip test.csv.zip -d test

Archive:  train.csv.zip
  inflating: train/train.csv         
Archive:  test.csv.zip
  inflating: test/test.csv           


In [4]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#importing necessary Python libraries
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


# Set the aesthetic style of the plots.
sns.set_style(style='darkgrid')
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
housing_df_train=pd.read_csv("train/train.csv")
housing_df_train.head(5)

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,CoverageField8,CoverageField9,CoverageField11A,CoverageField11B,SalesField1A,SalesField1B,SalesField2A,SalesField2B,SalesField3,SalesField4,SalesField5,SalesField6,SalesField7,SalesField8,SalesField9,SalesField10,SalesField11,SalesField12,...,GeographicField44A,GeographicField44B,GeographicField45A,GeographicField45B,GeographicField46A,GeographicField46B,GeographicField47A,GeographicField47B,GeographicField48A,GeographicField48B,GeographicField49A,GeographicField49B,GeographicField50A,GeographicField50B,GeographicField51A,GeographicField51B,GeographicField52A,GeographicField52B,GeographicField53A,GeographicField53B,GeographicField54A,GeographicField54B,GeographicField55A,GeographicField55B,GeographicField56A,GeographicField56B,GeographicField57A,GeographicField57B,GeographicField58A,GeographicField58B,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,23,0.9403,0.0006,965,1.0200,N,17,23,17,23,15,22,16,22,13,22,13,23,T,D,2,1,7,18,3,8,0,5,5,24,V,48649,0,0,0,0,...,8,4,20,22,10,8,6,5,15,13,19,18,16,14,21,23,21,23,16,11,22,24,7,14,-1,17,15,17,14,18,9,9,-1,8,-1,18,-1,10,N,CA
1,2,2014-04-22,0,F,7,1.0006,0.0040,548,1.2433,N,6,8,6,8,5,7,5,8,13,22,13,23,T,E,5,9,5,14,6,18,1,5,5,11,P,26778,0,0,1,1,...,23,24,11,15,21,24,6,11,21,21,18,15,20,20,13,12,12,12,15,9,13,11,11,20,-1,9,18,21,8,7,10,10,-1,11,-1,17,-1,20,N,NJ
2,4,2014-08-25,0,F,7,1.0006,0.0040,548,1.2433,N,7,12,7,12,6,10,7,11,25,25,13,23,T,J,4,6,3,10,4,11,1,5,5,11,K,8751,0,0,2,2,...,21,22,24,25,20,22,7,13,23,23,20,19,20,20,18,20,19,21,20,19,11,8,3,3,-1,5,21,24,12,15,15,18,-1,21,-1,11,-1,8,N,NJ
3,6,2013-04-15,0,J,10,0.9769,0.0004,"1,165",1.2665,N,3,2,3,2,2,2,3,2,13,22,13,23,Y,F,15,23,8,19,14,24,0,5,5,23,V,43854,0,0,0,0,...,3,1,14,22,6,2,7,14,11,8,19,18,18,16,13,12,13,12,17,13,5,2,3,4,-1,7,14,14,14,18,6,5,-1,10,-1,9,-1,21,N,TX
4,8,2014-01-25,0,E,23,0.9472,0.0006,"1,487",1.3045,N,8,13,8,13,7,11,7,13,13,22,13,23,T,F,4,6,3,6,3,6,1,5,5,7,R,12505,1,0,0,0,...,24,25,9,11,25,25,5,3,22,22,21,21,17,15,25,25,25,25,17,13,13,11,3,4,-1,7,11,9,10,10,18,22,-1,10,-1,11,-1,12,N,IL


In [6]:
housing_df_test=pd.read_csv("test/test.csv")
housing_df_test.head(5)

,QuoteNumber,Original_Quote_Date,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,CoverageField8,CoverageField9,CoverageField11A,CoverageField11B,SalesField1A,SalesField1B,SalesField2A,SalesField2B,SalesField3,SalesField4,SalesField5,SalesField6,SalesField7,SalesField8,SalesField9,SalesField10,SalesField11,SalesField12,SalesField13,...,GeographicField44A,GeographicField44B,GeographicField45A,GeographicField45B,GeographicField46A,GeographicField46B,GeographicField47A,GeographicField47B,GeographicField48A,GeographicField48B,GeographicField49A,GeographicField49B,GeographicField50A,GeographicField50B,GeographicField51A,GeographicField51B,GeographicField52A,GeographicField52B,GeographicField53A,GeographicField53B,GeographicField54A,GeographicField54B,GeographicField55A,GeographicField55B,GeographicField56A,GeographicField56B,GeographicField57A,GeographicField57B,GeographicField58A,GeographicField58B,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,3,2014-08-12,E,16,0.9364,0.0006,"1,487",1.3045,N,4,4,4,4,3,3,3,4,13,22,13,23,Y,K,13,22,6,16,9,21,0,5,5,11,P,67052,0,0,0,0,0,...,22,23,9,12,25,25,6,9,4,2,16,12,20,20,2,2,2,1,1,1,10,7,25,25,-1,19,19,22,12,15,1,1,-1,1,-1,20,-1,25,Y,IL
1,5,2013-09-07,F,11,0.9919,0.0038,564,1.1886,N,8,14,8,14,7,12,8,13,13,22,13,23,T,E,4,5,3,6,3,6,1,5,5,4,R,27288,1,0,0,0,0,...,23,24,12,21,23,25,7,11,16,14,13,6,17,15,7,5,7,5,13,7,14,14,7,14,-1,4,1,1,5,3,10,10,-1,5,-1,5,-1,21,N,NJ
2,7,2013-03-29,F,15,0.8945,0.0038,564,1.0670,N,11,18,11,18,10,16,10,18,13,22,13,23,T,E,3,3,5,14,3,9,1,5,5,23,V,65264,0,1,2,2,0,...,16,18,9,10,14,16,6,8,20,19,17,14,16,13,20,22,20,22,20,18,10,7,4,7,-1,11,13,12,18,22,10,11,-1,20,-1,22,-1,11,N,NJ
3,9,2015-03-21,K,21,0.8870,0.0004,"1,113",1.2665,Y,14,22,15,22,13,20,22,25,13,22,13,23,Y,F,5,9,9,20,5,16,1,5,5,11,R,32725,1,1,1,1,0,...,11,11,9,10,11,13,15,21,14,12,17,13,10,6,20,22,20,22,19,16,12,11,4,6,-1,13,10,8,5,3,8,8,-1,13,-1,8,-1,21,N,TX
4,10,2014-12-10,B,25,0.9153,0.0007,935,1.0200,N,4,5,4,5,4,4,4,5,13,22,13,23,Y,D,12,21,1,1,3,6,0,5,5,11,T,56025,0,1,1,1,0,...,9,8,25,25,9,3,9,18,7,4,16,12,13,9,8,6,8,6,11,5,19,21,13,21,-1,23,11,8,5,3,7,7,-1,3,-1,22,-1,21,N,CA


In [7]:
housing_df_train.drop(['PropertyField29','PersonalField84'], axis=1, inplace=True)
housing_df_test.drop(['PropertyField29','PersonalField84'], axis=1, inplace=True)

In [8]:
housing_df_train['Date'] = pd.to_datetime(pd.Series(housing_df_train['Original_Quote_Date']))

# Drop 'Original_Quote_date'
housing_df_train = housing_df_train.drop('Original_Quote_Date', axis=1)

# Extract year,month,weekday from 'Date'
housing_df_train['Year'] = housing_df_train['Date'].apply(lambda x: x.year)
housing_df_train['Month'] = housing_df_train['Date'].apply(lambda x: x.month)
housing_df_train['weekday'] = housing_df_train['Date'].apply(lambda x: x.weekday())
housing_df_train=housing_df_train.drop('Date', axis=1)

In [9]:
housing_df_test['Date'] = pd.to_datetime(pd.Series(housing_df_test['Original_Quote_Date']))

# Drop 'Original_Quote_date'
housing_df_test = housing_df_test.drop('Original_Quote_Date', axis=1)

# Extract year,month,weekday from 'Date'
housing_df_test['Year'] = housing_df_test['Date'].apply(lambda x: x.year)
housing_df_test['Month'] = housing_df_test['Date'].apply(lambda x: x.month)
housing_df_test['weekday'] = housing_df_test['Date'].apply(lambda x: x.weekday())
housing_df_test=housing_df_test.drop('Date', axis=1)

In [10]:
housing_df_train = housing_df_train.fillna(-1)
housing_df_test = housing_df_test.fillna(-1)


In [11]:
for f in housing_df_train.columns:
    if housing_df_train[f].dtype=='object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(housing_df_train[f].values) + list(housing_df_test[f].values))
        housing_df_train[f] = lbl.transform(list(housing_df_train[f].values))
        housing_df_test[f] = lbl.transform(list(housing_df_test[f].values))

Field6
Field10
Field12
CoverageField8
CoverageField9
SalesField7
PersonalField7
PersonalField16
PersonalField17
PersonalField18
PersonalField19
PropertyField3
PropertyField4
PropertyField5
PropertyField7
PropertyField14
PropertyField28
PropertyField30
PropertyField31
PropertyField32
PropertyField33
PropertyField34
PropertyField36
PropertyField37
PropertyField38
GeographicField63
GeographicField64


In [12]:
y_train=housing_df_train['QuoteConversion_Flag']
X_train = housing_df_train.drop(['QuoteConversion_Flag'], axis=1)

In [13]:
def get_roc_auc_score(model, flag=True):
    '''
    model : classifier to predict values of X

    '''
    c = []
    pred_train = model.predict(X_train)
    
    train_rec = metrics.roc_auc_score(y_train,pred_train)
    c.append(train_rec) # adding train recall to list
    if flag == True:
        print("ROC_AUC_Score on training set : ", train_rec)
        
    return c 

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [15]:
rf_estimator = RandomForestClassifier(random_state=1,oob_score=True,bootstrap=True, class_weight='balanced')
# Grid of parameters to choose from
params = {
              'n_estimators':[50,100,200],
              'max_features':[3,6,9],
              'min_samples_leaf':[20,40,60],
              'n_jobs':[-1]
}

# Run the grid search
grid_obj = GridSearchCV(rf_estimator, params,cv=5, scoring='roc_auc')
grid_obj = grid_obj.fit(X_train, y_train)

In [21]:
rf_estimator = grid_obj.best_estimator_
print(rf_estimator)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)


In [17]:
# Fit the best algorithm to the data. 
rf_estimator.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [19]:
get_roc_auc_score(rf_estimator)

ROC_AUC_Score on training set :  0.9082256545162408


[0.9082256545162408]

In [25]:
rf2= RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=15, max_features=68,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8,min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=446,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [26]:
rf2.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=15, max_features=68,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=446,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [27]:
get_roc_auc_score(rf2)

ROC_AUC_Score on training set :  0.9041808765350496


[0.9041808765350496]

In [28]:
! unzip sample_submission.csv.zip -d submission

Archive:  sample_submission.csv.zip
  inflating: submission/sample_submission.csv  


In [30]:
output_submission = pd.DataFrame(zip(housing_df_test.QuoteNumber,rf2.predict_proba(housing_df_test)[:,1]), columns = ['QuoteNumber','QuoteConversion_Flag'])
output_submission.to_csv('randomforest_with_hyperparameters.csv', index=False)